In [1]:
import pandas as pd
import numpy as np

In [2]:
# from importlib import reload # used for debugging

# local helper modules
from load_data import load_data
import data_prep as dprep

import Preparation

In [62]:
# used for debugging

# reload(Preparation)
# reload(dprep)

<module 'data_prep' from '/home/shsnail/PycharmProjects/HomePro/data_prep/__init__.py'>

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [63]:
prep = Preparation.Preparation()
clf = RandomForestClassifier()

pipeline = Pipeline(steps=[
    ('prep', prep),
    ('clf', clf),
])

# test model with train_test_split

In [ ]:
df = load_data('expo_train_data.csv')

In [20]:
y = df.loc[:, 'response_flag']
X = df.drop('response_flag', axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30
)

In [8]:
pipeline.fit(X_train, y_train)

fit_transform
(9800, 657)
filled


Pipeline(memory=None,
     steps=[('prep', <Preparation.Preparation object at 0x7fcb2cbc9208>), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [9]:
y_pred = pipeline.predict(X_test)
y_pred_prob = pipeline.predict_proba(X_test)

transform
transform


In [10]:
from sklearn.metrics import make_scorer, roc_auc_score

In [18]:
def scores(y_true, y_pred):
    auc = roc_auc_score(y_true, y_pred)
    
    y_true = y_true.astype(bool)
    y_pred = y_pred > 0.5
    
    TP = (y_pred & y_true).sum()
    precision = TP/y_pred.sum()
    recall = TP/y_true.sum()
    f1 = 2*precision*recall/(precision+recall)
    
    accuracy = (y_pred == y_true).mean()
    
    return precision, recall, f1, accuracy, auc

def accuracy(y_true, y_pred):
    acc = (y_pred == y_true).mean()
    return acc


In [12]:
ss = scores(y_test.values, y_pred_prob[:, 1])

In [13]:
print('''
precision: {:0.2f} %
   recall: {:0.2f} %
f-measure: {:0.2f} %

 accuracy: {:0.2f} %
      AUC: {:0.2f} %
'''.format(*map(lambda x: x*100, ss)))


precision: 51.37 %
   recall: 45.14 %
f-measure: 48.05 %

 accuracy: 52.69 %
      AUC: 53.88 %



# test model with cross validation

In [14]:
from sklearn.model_selection import cross_validate

In [64]:
scorers = dict(
    acc=make_scorer(accuracy),
    auc=make_scorer(roc_auc_score)
)
cv = cross_validate(pipeline, X, y, scoring=scorers)

fit_transform
(9332, 635)
filled
isnull False
transform
transform
transform
transform
fit_transform
(9334, 665)
filled
isnull False
transform
transform
transform
transform
fit_transform
(9334, 712)
filled
isnull False
transform
transform
transform
transform


In [65]:
cv

/home/shsnail/anaconda3/envs/datasci/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_acc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/shsnail/anaconda3/envs/datasci/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_auc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([32.48552918, 35.94692612, 61.8673842 ]),
 'score_time': array([0.36811638, 0.61666703, 0.45040417]),
 'test_acc': array([0.51949443, 0.53793399, 0.5327904 ]),
 'test_auc': array([0.51949443, 0.53793399, 0.5327904 ]),
 'train_acc': array([0.981033  , 0.98307264, 0.98167988]),
 'train_auc': array([0.981033  , 0.98307264, 0.98167988])}

# note

note that, by visualizing each tree used in random_forest, the model has only 10 trees with dramatic heights which mean the model is too overfit. to avoid the problem mentioned above, specifying the max_depth of the forest would be helpful.

the trees are visulized and are stored in directory: `model/random_forest01/`